# Import Statements and Magic Commands

In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import linear_model
from sklearn.metrics import mean_squared_log_error
# WHEN CHECKING THE MODEL: use as np.sqrt(mean_squared_log_error( y_test, predictions ))
from pandas.tseries.holiday import USFederalHolidayCalendar
%matplotlib inline

---
# Load the Data

### Building and Weather Data

In [6]:
# Note: These assume that the data has been saved to a subdirectory named "energy"
bldgData = pd.read_csv("energy/building_metadata.csv")
weatherTrain = pd.read_csv("energy/weather_train.csv")
weatherTest = pd.read_csv("energy/weather_test.csv")
allWeather = weatherTrain.append(weatherTest)

I'm combining the weather data into one dataframe. There's really no need for it to be separate. Plus, I have a hunch that the _current_ meter reading is better modeled as a function of the average of the _current_ weather data and the _last_ weather data. In other words, how much energy a building used in the last hour is based on the weather for the last hour, not necessarily the weather at the current timestamp. If this is the case, then the first prediction in the test data will need the last weather data from the training set.

### Training Data

In [7]:
energyTrain = pd.read_csv("energy/train.csv")

### Test Data
Warning: This will use a lot of memory! My virtual machine is sitting at 5.5 GB of RAM utilization right now.

In [8]:
energyTest = pd.read_csv("energy/test.csv")

### Explore the Data

In [4]:
#print(bldgData.head(5))
#print(allWeather.head(5))
#print(allWeather.tail(5))
print(energyTrain.head(5))

   building_id  meter            timestamp  meter_reading
0            0      0  2016-01-01 00:00:00            0.0
1            1      0  2016-01-01 00:00:00            0.0
2            2      0  2016-01-01 00:00:00            0.0
3            3      0  2016-01-01 00:00:00            0.0
4            4      0  2016-01-01 00:00:00            0.0


---
# 0. No Model

Just submit the "sample_submission.csv" and see what score that yields.

### Kaggle RMSLE score for No Model = 

---
# 1.a. The "Naïve Model" - Version 1

This model computes the average usage for a `(building_id, meter, hourly time)` tuple. This version of the Naïve Model is not aware of "workdays" vs. "non-workdays" (i.e. weekends and holidays).

**Train**

In [6]:
naiveModel = {}

for row in energyTrain.itertuples(index=False):
    building_id = str(row.building_id)
    meter = str(row.meter)
    time = row.timestamp.split(" ")[1]
    meter_reading = row.meter_reading
    
    key = building_id + "-" + meter + "-" + time
    
    if key in naiveModel:
        naiveModel[key][0] += meter_reading
        naiveModel[key][1] += 1.0
    else:
        naiveModel[key] = [meter_reading,1.0]
        
for key in naiveModel.keys():
    totalUsed = naiveModel[key][0]
    readingCount = naiveModel[key][1]
    naiveModel[key] = totalUsed / readingCount

**Predict**

In [7]:
# Note: Create a subdirectory named "results" before running
naiveResults = "row_id,meter_reading\n"

for row in energyTest.itertuples(index=False):
    row_id = str(row.row_id)
    building_id = str(row.building_id)
    meter = str(row.meter)
    time = row.timestamp.split(" ")[1]
    
    key = building_id + "-" + meter + "-" + time
    
    pred = naiveModel[key]

    naiveResults += row_id + "," + '{0:.4f}'.format(pred) + "\n"
    
outFile = open("results/naiveResults01.csv","w")
outFile.write(naiveResults)
outFile.close()

### Kaggle RMSLE score for the Naïve Method, Version 1 = 1.46

---
# 1.b. The "Naïve Model" - Version 2

This model improves on Version 1 by computing two averages for each `(building_id, meter, hourly time)` tuple: one for "workdays" and one for "non-workdays" (i.e. weekends and holidays).

**Train**

In [8]:
cal = USFederalHolidayCalendar() # This assumes all of the buildings are in the US, but they are not.
holidays = cal.holidays(start='2016-01-01', end='2018-12-31').to_pydatetime()

naiveModel = {}

for row in energyTrain.itertuples(index=False):
    building_id = str(row.building_id)
    meter = str(row.meter)
    dateStr, time = row.timestamp.split(" ")
    meter_reading = row.meter_reading
    
    date = datetime.datetime.strptime(dateStr, '%Y-%m-%d')
    
    if (date in holidays) or (date.weekday() > 4):
        workday = '0'
    else:
        workday = '1'
    
    key = workday + "-" + building_id + "-" + meter + "-" + time
    
    if key in naiveModel:
        naiveModel[key][0] += meter_reading
        naiveModel[key][1] += 1.0
    else:
        naiveModel[key] = [meter_reading,1.0] 
        
for key in naiveModel.keys():
    totalUsed = naiveModel[key][0]
    readingCount = naiveModel[key][1]
    naiveModel[key] = totalUsed / readingCount

**Predict**

In [9]:
# Note: Create a subdirectory named "results" before running
naiveResults = "row_id,meter_reading\n"

for row in energyTest.itertuples(index=False):
    row_id = str(row.row_id)
    building_id = str(row.building_id)
    meter = str(row.meter)
    dateStr, time = row.timestamp.split(" ")
    
    date = datetime.datetime.strptime(dateStr, '%Y-%m-%d')
    
    if (date in holidays) or (date.weekday() > 4):
        workday = '0'
    else:
        workday = '1'
    
    key = workday + "-" + building_id + "-" + meter + "-" + time
    
    pred = naiveModel[key]

    naiveResults += row_id + "," + '{0:.4f}'.format(pred) + "\n"
    
outFile = open("results/naiveResults02.csv","w")
outFile.write(naiveResults)
outFile.close()

### Kaggle RMSLE score for the Naïve Method, Version 2 = 1.45

---
# Intermission: Merge Training Data prior to Machine Learning

In [8]:
trainJoinBldg = pd.merge(energyTrain, bldgData, on='building_id')
fullTraining = pd.merge(trainJoinBldg, allWeather, on=['site_id', 'timestamp'])

del energyTrain
del trainJoinBldg
del bldgData
del weatherTrain
del weatherTest
del allWeather

In [9]:
print(fullTraining.shape)
fullTraining = fullTraining[fullTraining.meter_reading != 0.0]
print(fullTraining.shape)

slimTraining = fullTraining[['building_id', 'meter', 'timestamp', 'meter_reading']]

del fullTraining

print(slimTraining.shape)

(20125605, 16)
(18257718, 16)
(18257718, 5)


In [10]:
slimTraining["hour"] = ""
slimTraining["month"] = ""
slimTraining["dayOfWeek"] = ""

for i, row in slimTraining.iterrows():
    slimTraining.at[i,'hour'] = row['timestamp'].split(" ")[1]
    date = datetime.datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')
    slimTraining.at[i,'month'] = date.strftime('%b')
    slimTraining.at[i,'dayOfWeek'] = str(date.weekday())
    
slimTraining.drop("timestamp", axis=1, inplace=True)

In [13]:
print(slimTraining.shape)
slimTraining.to_csv("custom/hourMonthDayofweek.csv")

(18257718, 7)


In [2]:
slimTraining = pd.read_csv("custom/hourMonthDayofweek.csv")
slimTraining.head(5)

,Unnamed: 0,building_id,meter,meter_reading,air_temperature,hour,month,dayOfWeek
0,45,46,0,53.2397,25.0,00:00:00,Jan,4
1,72,74,0,43.0013,25.0,00:00:00,Jan,4
2,91,93,0,52.4206,25.0,00:00:00,Jan,4
3,123,20,0,91.9886,24.4,01:00:00,Jan,4
4,148,46,0,53.6492,24.4,01:00:00,Jan,4


In [3]:
buildingMeters = []

for row in slimTraining.itertuples(index=False):
    if (row.building_id, row.meter) not in buildingMeters:
        buildingMeters.append((row.building_id, row.meter))
    
print(len(buildingMeters))

2380


In [4]:
readyToTrain = pd.get_dummies(slimTraining, columns=['hour', 'month', 'dayOfWeek'])

In [5]:
readyToTrain.head(5)

,Unnamed: 0,building_id,meter,meter_reading,air_temperature,hour_00:00:00,hour_01:00:00,hour_02:00:00,hour_03:00:00,hour_04:00:00,...,month_Nov,month_Oct,month_Sep,dayOfWeek_0,dayOfWeek_1,dayOfWeek_2,dayOfWeek_3,dayOfWeek_4,dayOfWeek_5,dayOfWeek_6
0,45,46,0,53.2397,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,72,74,0,43.0013,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,91,93,0,52.4206,25.0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,123,20,0,91.9886,24.4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,148,46,0,53.6492,24.4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
trainedModels = {}

counter = 0
oldPercent = 0.
for building_id, meter in buildingMeters:
    buildingMeterData = readyToTrain[(readyToTrain['building_id'] == building_id) & (readyToTrain['meter'] == meter)]
    # TRAIN A MODEL USING buildingMeterData
    y = buildingMeterData['meter_reading']
    X = buildingMeterData[[
        'hour_00:00:00',
        'hour_01:00:00',
        'hour_02:00:00',
        'hour_03:00:00',
        'hour_04:00:00',
        'hour_05:00:00',
        'hour_06:00:00',
        'hour_07:00:00',
        'hour_08:00:00',
        'hour_09:00:00',
        'hour_10:00:00',
        'hour_11:00:00',
        'hour_12:00:00',
        'hour_13:00:00',
        'hour_14:00:00',
        'hour_15:00:00',
        'hour_16:00:00',
        'hour_17:00:00',
        'hour_18:00:00',
        'hour_19:00:00',
        'hour_20:00:00',
        'hour_21:00:00',
        'hour_22:00:00',
        'hour_23:00:00',
        'month_Jan',
        'month_Feb',
        'month_Mar',
        'month_Apr',
        'month_May',
        'month_Jun',
        'month_Jul',
        'month_Aug',
        'month_Sep',
        'month_Oct',
        'month_Nov',
        'month_Dec',
        'dayOfWeek_0',
        'dayOfWeek_1',
        'dayOfWeek_2',
        'dayOfWeek_3',
        'dayOfWeek_4',
        'dayOfWeek_5',
        'dayOfWeek_6'
    ]]
    dtRegressor = tree.DecisionTreeRegressor()
    trainedModels[(building_id, meter)] = dtRegressor.fit(X, y)
    counter += 1
    percent = 100. * counter / 2380
    if percent - oldPercent > 5:
        print('{0:.2f}'.format(percent))
        oldPercent = percent

print(len(trainedModels))

5.04
10.08
15.13
20.17
25.21
30.25
35.29
40.34
45.38
50.42
55.46
60.50
65.55
70.59
75.63
80.67
85.71
90.76
95.80
2380


In [7]:
#print(trainedModels[(93,0)].feature_importances_)
del slimTraining
del readyToTrain

In [9]:
print(energyTest.shape)

(41697600, 4)


In [14]:
# Note: Create a subdirectory named "results" before running
dtrResults = "row_id,meter_reading\n"

counter = 0
oldPercent = 0.
for row in energyTest.itertuples(index=False):
    row_id = str(row.row_id)
    building_id = row.building_id
    meter = row.meter
    dateStr, hour = row.timestamp.split(" ")
    
    date = datetime.datetime.strptime(dateStr, '%Y-%m-%d')
    
    month = date.strftime('%b')
    dayOfWeek = str(date.weekday())
    
    features = pd.Series(0, index=[
        'hour_00:00:00',
        'hour_01:00:00',
        'hour_02:00:00',
        'hour_03:00:00',
        'hour_04:00:00',
        'hour_05:00:00',
        'hour_06:00:00',
        'hour_07:00:00',
        'hour_08:00:00',
        'hour_09:00:00',
        'hour_10:00:00',
        'hour_11:00:00',
        'hour_12:00:00',
        'hour_13:00:00',
        'hour_14:00:00',
        'hour_15:00:00',
        'hour_16:00:00',
        'hour_17:00:00',
        'hour_18:00:00',
        'hour_19:00:00',
        'hour_20:00:00',
        'hour_21:00:00',
        'hour_22:00:00',
        'hour_23:00:00',
        'month_Jan',
        'month_Feb',
        'month_Mar',
        'month_Apr',
        'month_May',
        'month_Jun',
        'month_Jul',
        'month_Aug',
        'month_Sep',
        'month_Oct',
        'month_Nov',
        'month_Dec',
        'dayOfWeek_0',
        'dayOfWeek_1',
        'dayOfWeek_2',
        'dayOfWeek_3',
        'dayOfWeek_4',
        'dayOfWeek_5',
        'dayOfWeek_6'
    ])
    
    features['hour_' + hour] = 1
    features['month_' + month] = 1
    features['dayOfWeek_' + dayOfWeek] = 1
    
    pred = trainedModels[(building_id, meter)].predict([features])[0]

    dtrResults += row_id + "," + '{0:.4f}'.format(pred) + "\n"
    
    counter += 1
    percent = 100. * counter / 41697600
    if percent - oldPercent > 5:
        print('{0:.2f}'.format(percent))
        oldPercent = percent
    
outFile = open("results/decisionTreeRegressor.csv","w")
outFile.write(dtrResults)
outFile.close()

5.00
10.00
15.00
20.00
25.00
30.00
35.00
40.00
45.00
50.00
55.00
60.00
65.00
70.00
75.00
80.00
85.00
90.00
95.00


In [7]:
# Explore new dataframes
#print(energyTrain.shape)
#print(trainJoinBldg.shape)
#thrownOut = energyTrain.shape[0] - trainJoinBldg.shape[0]
#thrownOutPercent = 100.* thrownOut * 1./ energyTrain.shape[0]
#print("% thrown out by merging with building data: " + '{0:.2f}'.format(thrownOutPercent) + "%")
#print(fullTraining.shape)
#thrownOut = trainJoinBldg.shape[0] - fullTraining.shape[0]
#thrownOutPercent = 100.* thrownOut * 1./ trainJoinBldg.shape[0]
#print("% thrown out by merging with weather data: " + '{0:.2f}'.format(thrownOutPercent) + "%")
#print(trainJoinBldg.head(5))
#print(fullTraining.head(5))
print(fullTraining.columns)

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed'],
      dtype='object')


`fullTraining` now has all the provided training data merged into one dataframe. i.e. Each row contains the meter reading, building info, and weather for that site+timestamp all in one row.

Only 0.45% of the rows get lost when merging with the weather data. This implies that not every site+timestamp is present in the weather data.

I assume it will also be necessary to repeat these steps for the test data prior to making predictions; however, if weather data is not present for a given site+timestamp, we cannot just drop the row - we must still make a prediction.

---
# Intermission: Manipulate the Data

In [5]:
# Define functions that create new columns based on data from other columns
def getMonth(row):
    date = datetime.datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S')
    return date.strftime('%b')

In [5]:
elecTrain = fullTraining[fullTraining['meter'] == 0]
chillWaterTrain = fullTraining[fullTraining['meter'] == 1]
steamTrain = fullTraining[fullTraining['meter'] == 2]
hotWaterTrain = fullTraining[fullTraining['meter'] == 3]

del fullTraining

In [8]:
# Create new columns
# elecTrain['month'] = elecTrain.apply(lambda row: getMonth(row), axis=1) ## Lambda way
elecTrain["month"] = ""
for i, row in elecTrain.iterrows():
     elecTrain.at[i,'month'] = datetime.datetime.strptime(row['timestamp'], '%Y-%m-%d %H:%M:%S').strftime('%b')

In [9]:
print(elecTrain.head(5))

   building_id  meter            timestamp  meter_reading  site_id  \
0            0      0  2016-01-01 00:00:00            0.0        0   
1            1      0  2016-01-01 00:00:00            0.0        0   
2            2      0  2016-01-01 00:00:00            0.0        0   
3            3      0  2016-01-01 00:00:00            0.0        0   
4            4      0  2016-01-01 00:00:00            0.0        0   

  primary_use  square_feet  year_built  floor_count  air_temperature  \
0   Education         7432      2008.0          NaN             25.0   
1   Education         2720      2004.0          NaN             25.0   
2   Education         5376      1991.0          NaN             25.0   
3   Education        23685      2002.0          NaN             25.0   
4   Education       116607      1975.0          NaN             25.0   

   cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
0             6.0             20.0                NaN              10

---
# 2. Linear Regression

I have a hunch that we'll need separate models for workdays and holidays/weekends.
So, we may end up needing a model for each of the following:

* workday / electricity
* workday / chilledwater
* workday / steam
* workday / hotwater
* non-workday / electricity
* non-workday / chilledwater
* non-workday / steam
* non-workday / hotwater

And we could maybe even add the `primary_use` as yet another variable if we really want/need to.